<a href="https://colab.research.google.com/github/VICTORNTA/Incomplete_project/blob/main/modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dropout,Dense
from tensorflow.keras.models import  Sequential
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
base_model = VGG16(input_shape=(150,150,3),include_top=False,weights='imagenet')

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_image_generator =ImageDataGenerator(
    rescale=1./255, 
    horizontal_flip=True,
    rotation_range = 120

    )

In [ ]:
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
batch_size = 128

In [ ]:
train_dir = r"/content/drive/MyDrive/kaggle/cars_train/classes"
validation_dir = r"/content/drive/MyDrive/kaggle/cars_test/split_test_classes"

In [ ]:
image_height = 150
image_width = 150

In [ ]:
validation_data_gen= validation_image_generator.flow_from_directory(
    batch_size = 64,
    directory = validation_dir,
    target_size = (image_height,image_width),
    class_mode = 'categorical',
)

Found 1960 images belonging to 196 classes.


In [ ]:
train_data_gen = train_image_generator.flow_from_directory(
    batch_size = batch_size,
    directory = train_dir,
    target_size = (image_height,image_width),
    class_mode = 'categorical',
)

Found 8144 images belonging to 196 classes.


In [ ]:
 sample_train_image,c = next(validation_data_gen)

In [ ]:
# import matplotlib.pyplot as plt

In [ ]:
# fig,axes = plt.subplots(5,5,figsize=(20,20))
# axes = axes.flatten()
# for _,img in zip(axes,sample_train_image[25:56]):
#   _.imshow(img) 
#   _.axis("off")

In [ ]:
mod = Flatten()(base_model.output)
mod = Dense(512,activation='relu')(mod)
mod = Dropout(0.5)(mod)
mod = Dense(196,activation =  "softmax")(mod)
model = tf.keras.models.Model(base_model.input,mod)

In [ ]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss')

In [ ]:
model.compile(
    optimizer ='adam',
    loss = "categorical_crossentropy",
   metrics = ['accuracy'],

)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [ ]:
model.fit_generator(
    train_data_gen,
    steps_per_epoch = 8144//128,
    epochs = 50,
    validation_data = validation_data_gen,
    )

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
63/63 [==============================] - 2228s 35s/step - loss: 5.3333 - accuracy: 0.0047 - val_loss: 5.2712 - val_accuracy: 0.0077
Epoch 2/50
48/63 [=====================>........] - ETA: 6:52 - loss: 5.2603 - accuracy: 0.0108

In [ ]:
  d = model.history.history

In [ ]:
d.keys()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
acc_df = pd.DataFrame(d)

In [ ]:
import seaborn as sns
sns.lineplot(x = acc_df["accuracy"] ,y = acc_df["loss"],)